## Import Libraries

In [1]:
# import standard libraries for project
import numpy as np
import pandas as pd 
from io import StringIO
from sklearn import preprocessing
from pandas import ExcelWriter
from datetime import datetime
 
from os import listdir
from os.path import isfile, join

import openpyxl

import math

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

import scipy

# show all rows and cols when printing output to terminal
pd.set_option('display.max_rows', 750, 'display.max_columns', 500)  

## Import Raw Data

In [2]:
# import raw data
import_path = './jupyter_data/'
all_data_raw = pd.read_csv(import_path +'joined_data.csv')

## Get Raw Expense Data from Joined DataFrame

#### Total Revenue and Expenses

In [3]:
# get "total revenue" by adding up subcategories for districts, just use total column for charters
    # "total" column is blank for districts, sub-categories are blank for charters. adding 
    # all revenue fields together doesn't result in double counting
    # exclude food-service-related revenue

m_and_o_rev = all_data_raw.loc[:,'Maintenance_and_Operation_Fund_Revenue'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
classroom_site_rev = all_data_raw.loc[:,'Classroom_Site_Project_Revenue'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
instructional_improvement_rev = all_data_raw.loc[:,'Instructional_Improvement_Project_Revenue'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
ell_rev = all_data_raw.loc[:,'English_Language_Learner_Project_Revenue'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
compensatory_instruction_rev = all_data_raw.loc[:,'Compensatory_Instruction_Project_Revenue'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
federal_and_state_proj_rev = all_data_raw.loc[:,'Federal_and_State_projects_Revenue'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
total_rev_all_sources_charter = all_data_raw.loc[:,'Total_Revenue_All_Sources'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

total_revenue = (m_and_o_rev 
                  + classroom_site_rev 
                  + instructional_improvement_rev 
                  + ell_rev 
                  + compensatory_instruction_rev 
                  + federal_and_state_proj_rev
                  + total_rev_all_sources_charter)

In [4]:
# get "total expenses" by adding up subcategories (necessary to align district and charter accounting)
    # exclude Food Expenses - very few charters (if any) provide food

m_and_o_exp = all_data_raw.loc[:,'Maintenance_and_Operation_Fund_Expenses'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
classroom_site_exp = all_data_raw.loc[:,'Classroom_Site_Project_Expenses'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
instructional_improvement_exp = all_data_raw.loc[:,'Instructional_Improvement_Project_Expenses'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
ell_exp = all_data_raw.loc[:,'English_Language_Learner_Project_Expenses'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
compensatory_instruction_exp = all_data_raw.loc[:,'Compensatory_Instruction_Project_Expenses'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
federal_and_state_proj_exp = all_data_raw.loc[:,'Federal_and_State_projects_Expenses'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

# NOTE: excludes "Student Success" expenses, which are in 2015 only
total_expenses = (m_and_o_exp 
                  + classroom_site_exp 
                  + instructional_improvement_exp
                  + ell_exp
                  + compensatory_instruction_exp 
                  + federal_and_state_proj_exp)

#### Expense Data by FUNCTION CODE

In [5]:
# salary instructional expenses (acct 1000 - includes 6100, 6200, 6300, 6400, 6500, 6800)

instruction_1000_regular = all_data_raw.loc[:,'Total_actual_100_Regular_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
instruction_1000_sped = all_data_raw.loc[:,'Total_actual_200_Special_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

instruction_1000_reg_sped = instruction_1000_regular + instruction_1000_sped

In [6]:
# support services for students (acct 2100)

support_serv_students_2100_regular = all_data_raw.loc[:,'Total_actual_100_Regular_education_2000_Support_services_2100_Students'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
support_serv_students_2100_sped = all_data_raw.loc[:,'Total_actual_200_Special_education_2000_Support_services_2100_Students'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

support_serv_students_2100_reg_sped = support_serv_students_2100_regular + support_serv_students_2100_sped

In [7]:
# support services for instruction (acct 2200)

support_serv_instruction_2200_regular = all_data_raw.loc[:,'Total_actual_100_Regular_education_2000_Support_services_2200_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
support_serv_instruction_2200_sped = all_data_raw.loc[:,'Total_actual_200_Special_education_2000_Support_services_2200_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

support_serv_instruction_2200_reg_sped = support_serv_instruction_2200_regular + support_serv_instruction_2200_sped

In [8]:
# admin-related expenses (accts 2300, 2400, 2500)

support_serv_general_admin_2300_regular = all_data_raw.loc[:,'Total_actual_100_Regular_education_2000_Support_services_2300_General_administration'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
support_serv_school_admin_2400_regular = all_data_raw.loc[:,'Total_actual_100_Regular_education_2000_Support_services_2400_School_administration'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
support_serv_central_serv_2500_regular = all_data_raw.loc[:,'Total_actual_100_Regular_education_2000_Support_services_2500_Central_services'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
support_serv_general_admin_2300_sped = all_data_raw.loc[:,'Total_actual_200_Special_education_2000_Support_services_2300_General_administration'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
support_serv_school_admin_2400_sped = all_data_raw.loc[:,'Total_actual_200_Special_education_2000_Support_services_2400_School_administration'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
support_serv_central_serv_2500_sped = all_data_raw.loc[:,'Total_actual_200_Special_education_2000_Support_services_2500_Central_services'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

support_serv_general_admin_2300_reg_sped = support_serv_general_admin_2300_regular + support_serv_general_admin_2300_sped
support_serv_school_admin_2400_reg_sped = support_serv_school_admin_2400_regular + support_serv_school_admin_2400_sped
support_serv_central_serv_2500_reg_sped = support_serv_central_serv_2500_regular + support_serv_central_serv_2500_sped

admin_expenses_all = (support_serv_general_admin_2300_reg_sped
                      + support_serv_school_admin_2400_reg_sped 
                      + support_serv_central_serv_2500_reg_sped)

In [9]:
# operation and maintenance of plant expenses (acct 2600)

o_and_m_plant_2600_regular = all_data_raw.loc[:,'Total_actual_100_Regular_education_2000_Support_services_2600_Operation_maintenance_of_plant'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
o_and_m_plant_2600_sped = all_data_raw.loc[:,'Total_actual_200_Special_education_2000_Support_services_2600_Operation_maintenance_of_plant'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

o_and_m_plant_2600_reg_sped = o_and_m_plant_2600_regular + o_and_m_plant_2600_sped

In [10]:
# total building-related expenses (accts 2600, 3000, 4000)

a = all_data_raw.loc[:,'Total_actual_100_Regular_education_2000_Support_services_2600_Operation_maintenance_of_plant']
b = all_data_raw.loc[:,'Total_actual_100_Regular_education_3000_Operation_of_noninstructional_services']
c = all_data_raw.loc[:,'Total_actual_100_Regular_education_4000_Facilities_acquisition_construction']
d = all_data_raw.loc[:,'Total_actual_200_Special_education_2000_Support_services_2600_Operation_maintenance_of_plant']
e = all_data_raw.loc[:,'Total_actual_200_Special_education_3000_Operation_of_noninstructional_services']
f = all_data_raw.loc[:,'Total_actual_200_Special_education_4000_Facilities_acquisition_construction']

building_related_expenses = a + b + c + d + e + f



In [11]:
# regular education expenses (does not include o&m or facilities-related expenses allocated to sped)

regular_exp_all = (instruction_1000_regular
                + support_serv_students_2100_regular
                + support_serv_instruction_2200_regular
                + support_serv_general_admin_2300_regular
                + support_serv_school_admin_2400_regular
                + support_serv_central_serv_2500_regular)

In [12]:
# sped education expenses (does not include o&m or facilities-related expenses allocated to sped)

sped_exp_all = (instruction_1000_sped
                + support_serv_students_2100_sped
                + support_serv_instruction_2200_sped
                + support_serv_general_admin_2300_sped 
                + support_serv_school_admin_2400_sped 
                + support_serv_central_serv_2500_sped)

#### Expense Data by OBJECT CODE

In [13]:
# make list of column names for non-inflation expenses

# 'Cash' = first col
# 'Other_6800_Classroom_Site_Project' = last col

start_col = all_data_raw.columns.get_loc('Cash')
end_col = all_data_raw.columns.get_loc('Other_6800_Classroom_Site_Project') + 1

data_cols = all_data_raw.columns[start_col:end_col]
expense_table_nominal = all_data_raw.iloc[:,start_col:end_col]

In [14]:
# salaries expenses, account 6100
salaries_cols_idx = []
salaries_acct_txt = '6100'

for i in range(0,len(data_cols)):
    if (salaries_acct_txt in data_cols[i]):
        salaries_cols_idx.append(i)
        
salaries_6100_exp = expense_table_nominal.iloc[:,salaries_cols_idx].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True).sum(axis=1)

In [15]:
# employee benefits expenses, account 6200
benefits_cols_idx = []
benefits_acct_txt = '6200'

for i in range(0,len(data_cols)):
    if (benefits_acct_txt in data_cols[i]):
        benefits_cols_idx.append(i)
        
benefits_6200_exp = expense_table_nominal.iloc[:,benefits_cols_idx].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True).sum(axis=1)

In [16]:
# sum of account 6100 (all salaries) and account 6200 (all benefits)
all_salaries_benefits_6100_6200 = salaries_6100_exp + benefits_6200_exp

In [17]:
# purchased expenses, account 6300 (6400 and 6500 too)
purch_serv_cols_idx = []
purch_serv_acct_txt = '6300'

for i in range(0,len(data_cols)):
    if (purch_serv_acct_txt in data_cols[i]):
        purch_serv_cols_idx.append(i)
        
purch_serv_6300_6400_6500_exp = expense_table_nominal.iloc[:,purch_serv_cols_idx].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True).sum(axis=1)

In [18]:
# supplies expenses, account 6600
supplies_cols_idx = []
supplies_acct_txt = '6600'

for i in range(0,len(data_cols)):
    if (supplies_acct_txt in data_cols[i]):
        supplies_cols_idx.append(i)
        
supplies_6600_exp = expense_table_nominal.iloc[:,supplies_cols_idx].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True).sum(axis=1)

In [19]:
# other expenses, account 6800
other_exp_cols_idx = []
other_exp_acct_txt = '6800'

for i in range(0,len(data_cols)):
    if (other_exp_acct_txt in data_cols[i]):
        other_exp_cols_idx.append(i)
        
other_6800_exp = expense_table_nominal.iloc[:,other_exp_cols_idx].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True).sum(axis=1)

#### Expense Data by FUNCTION CODE & OBJECT CODE

In [20]:
# salary instructional expenses (no benefits) (acct 1000, 6100 ONLY, i.e. salaries only)

salary_instruction_1000_regular_6100 = all_data_raw.loc[:,'Salaries_6100_100_Regular_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
salary_instruction_1000_sped_6100 = all_data_raw.loc[:,'Salaries_6100_200_Special_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
 
salary_instruction_1000_6100_reg_sped = salary_instruction_1000_regular_6100 + salary_instruction_1000_sped_6100

In [21]:
# instruction benefits expenses (acct 1000, 6200 ONLY, i.e. benefits only)

benefits_instruction_1000_regular_6200 = all_data_raw.loc[:,'Employee_benefits_6200_100_Regular_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
benefits_instruction_1000_sped_6200 = all_data_raw.loc[:,'Employee_benefits_6200_200_Special_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

benefits_instruction_1000_6200_reg_sped = benefits_instruction_1000_regular_6200 + benefits_instruction_1000_sped_6200

In [22]:
# all salaries and benefits related to instruction for regular and sped

salary_benefits_instruction_1000_6100_6200_reg_sped = salary_instruction_1000_6100_reg_sped + benefits_instruction_1000_6200_reg_sped

In [23]:
# all salaries and benefits related to instruction for regular and sped AND classroom site fund expenses

salary_benefits_instruction_1000_6100_6200_reg_sped_class_site = classroom_site_exp + salary_benefits_instruction_1000_6100_6200_reg_sped

In [24]:
# instruction-related presonnel including purchased services and support services accts 2100 and 2200

a = all_data_raw.loc[:, 'Salaries_6100_100_Regular_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
b = all_data_raw.loc[:, 'Salaries_6100_100_Regular_education_2000_Support_services_2100_Students'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
c = all_data_raw.loc[:, 'Salaries_6100_100_Regular_education_2000_Support_services_2200_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

d = all_data_raw.loc[:, 'Salaries_6100_200_Special_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
e = all_data_raw.loc[:, 'Salaries_6100_200_Special_education_2000_Support_services_2100_Students'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
f = all_data_raw.loc[:, 'Salaries_6100_200_Special_education_2000_Support_services_2200_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)


g = all_data_raw.loc[:, 'Employee_benefits_6200_100_Regular_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
h = all_data_raw.loc[:, 'Employee_benefits_6200_100_Regular_education_2000_Support_services_2100_Students'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
i = all_data_raw.loc[:, 'Employee_benefits_6200_100_Regular_education_2000_Support_services_2200_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

j = all_data_raw.loc[:, 'Employee_benefits_6200_200_Special_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
k = all_data_raw.loc[:, 'Employee_benefits_6200_200_Special_education_2000_Support_services_2100_Students'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
l = all_data_raw.loc[:, 'Employee_benefits_6200_200_Special_education_2000_Support_services_2200_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

m = all_data_raw.loc[:, 'Purchased_services_6300_6400_6500_100_Regular_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
n = all_data_raw.loc[:, 'Purchased_services_6300_6400_6500_100_Regular_education_2000_Support_services_2100_Students'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
o = all_data_raw.loc[:, 'Purchased_services_6300_6400_6500_100_Regular_education_2000_Support_services_2200_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

p = all_data_raw.loc[:, 'Purchased_services_6300_6400_6500_200_Special_education_1000_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
q = all_data_raw.loc[:, 'Purchased_services_6300_6400_6500_200_Special_education_2000_Support_services_2100_Students'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
r = all_data_raw.loc[:, 'Purchased_services_6300_6400_6500_200_Special_education_2000_Support_services_2200_Instruction'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

s = classroom_site_exp

total_instruction_related_personnel = a + b + c + d + e + f + g + h + i + j + k + l + m + n + o + p + q + r + s

In [25]:
total_instruction_related_personnel

0         283799.0
1         373375.0
2         626520.0
3        1515935.0
4         463963.0
           ...    
2693     1995240.0
2694      343612.0
2695    31945581.0
2696      594480.0
2697    40935008.0
Length: 2698, dtype: float64

#### Additional Data from Annual Financial Report

In [26]:
all_data_raw.columns

Index(['lea_year_key', 'lea_id_x', 'lea_name_x', 'ctds_id_str', 'year_x',
       'asian', 'american_indian_alaskan', 'black', 'hispanic', 'white',
       ...
       '7', '8', '9', '10', '11', '12', 'Total_x', 'Female', 'Male',
       'Total_y'],
      dtype='object', length=1374)

In [27]:
# ctds id
ctds_id = all_data_raw.loc[:, 'ctds_id_str']

year = all_data_raw.loc[:, 'Year']

In [28]:
ctds_id

0       x138761000
1       x078793000
2       x078542000
3       x108734000
4       x088704000
           ...    
2693    x088755000
2694    x080313000
2695    x140401000
2696    x148758000
2697    x140570000
Name: ctds_id_str, Length: 2698, dtype: object

In [29]:
# cash balance (only charters have this data)
cash_balance = all_data_raw.loc[:, 'Cash'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

In [30]:
# pull out student, teacher, and avg salary data

total_students = all_data_raw.loc[:,'Total_x'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

certified_teachers = all_data_raw.loc[:, 'Certified_FTE_Teachers'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
non_certified_teachers = all_data_raw.loc[:, 'NonCertified_FTE_Teachers'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)
contract_teachers = all_data_raw.loc[:, 'Contract_FTE_Teachers'].apply(pd.to_numeric, errors='coerce').fillna(0).reset_index(drop=True)

total_teachers = certified_teachers + non_certified_teachers + contract_teachers

avg_teacher_salary = all_data_raw['Avg_Teacher_Salary']
avg_teacher_salary_infl = all_data_raw['Avg_Teacher_Salary_inflation']

In [31]:
# make FRL % column
num = all_data_raw['freereduced']
den = all_data_raw['all_students']
pct_free_reduced = num / den

# make % non-white colum
num_1 = all_data_raw['white']
den_1 = all_data_raw['total']
pct_white = num_1 / den_1
pct_non_white = (den_1 - num_1) / den_1

# make % students with disabilities column
num_2 = all_data_raw['swd']
den_2 = all_data_raw['total']
pct_swd = num_2 / den_2

In [32]:
# initialize dataframe
afr_data_df=pd.DataFrame()

afr_data_df=afr_data_df.assign(ctds_id=ctds_id)
afr_data_df=afr_data_df.assign(year_a=year)

afr_data_df=afr_data_df.assign(m_and_o_rev=m_and_o_rev) 
afr_data_df=afr_data_df.assign(classroom_site_rev=classroom_site_rev)
afr_data_df=afr_data_df.assign(instructional_improvement_rev=instructional_improvement_rev)
afr_data_df=afr_data_df.assign(ell_rev=ell_rev)
afr_data_df=afr_data_df.assign(compensatory_instruction_rev=compensatory_instruction_rev) 
afr_data_df=afr_data_df.assign(federal_and_state_proj_rev=federal_and_state_proj_rev)
afr_data_df=afr_data_df.assign(total_rev_all_sources_charter=total_rev_all_sources_charter)
afr_data_df=afr_data_df.assign(total_revenue=total_revenue)

afr_data_df=afr_data_df.assign(m_and_o_exp=m_and_o_exp)
afr_data_df=afr_data_df.assign(classroom_site_exp=classroom_site_exp)
afr_data_df=afr_data_df.assign(instructional_improvement_exp=instructional_improvement_exp)
afr_data_df=afr_data_df.assign(ell_exp=ell_exp)
afr_data_df=afr_data_df.assign(compensatory_instruction_exp=compensatory_instruction_exp)
afr_data_df=afr_data_df.assign(federal_and_state_proj_exp=federal_and_state_proj_exp)
afr_data_df=afr_data_df.assign(total_expenses=total_expenses)

afr_data_df=afr_data_df.assign(instruction_1000_regular=instruction_1000_regular)
afr_data_df=afr_data_df.assign(instruction_1000_sped=instruction_1000_sped)
afr_data_df=afr_data_df.assign(instruction_1000_reg_sped=instruction_1000_reg_sped)
afr_data_df=afr_data_df.assign(support_serv_students_2100_regular=support_serv_students_2100_regular)
afr_data_df=afr_data_df.assign(support_serv_students_2100_sped=support_serv_students_2100_sped)
afr_data_df=afr_data_df.assign(support_serv_students_2100_reg_sped=support_serv_students_2100_reg_sped)
afr_data_df=afr_data_df.assign(support_serv_instruction_2200_regular=support_serv_instruction_2200_regular)
afr_data_df=afr_data_df.assign(support_serv_instruction_2200_sped=support_serv_instruction_2200_sped)
afr_data_df=afr_data_df.assign(support_serv_instruction_2200_reg_sped=support_serv_instruction_2200_reg_sped)
afr_data_df=afr_data_df.assign(support_serv_general_admin_2300_regular=support_serv_general_admin_2300_regular)
afr_data_df=afr_data_df.assign(support_serv_general_admin_2300_sped=support_serv_general_admin_2300_sped)
afr_data_df=afr_data_df.assign(support_serv_general_admin_2300_reg_sped=support_serv_general_admin_2300_reg_sped)
afr_data_df=afr_data_df.assign(support_serv_school_admin_2400_regular=support_serv_school_admin_2400_regular)
afr_data_df=afr_data_df.assign(support_serv_school_admin_2400_sped=support_serv_school_admin_2400_sped)
afr_data_df=afr_data_df.assign(support_serv_school_admin_2400_reg_sped=support_serv_school_admin_2400_reg_sped)
afr_data_df=afr_data_df.assign(support_serv_central_serv_2500_regular=support_serv_central_serv_2500_regular)
afr_data_df=afr_data_df.assign(support_serv_central_serv_2500_sped=support_serv_central_serv_2500_sped)
afr_data_df=afr_data_df.assign(support_serv_central_serv_2500_reg_sped=support_serv_central_serv_2500_reg_sped)
afr_data_df=afr_data_df.assign(admin_expenses_all=admin_expenses_all)
afr_data_df=afr_data_df.assign(o_and_m_plant_2600_regular=o_and_m_plant_2600_regular)
afr_data_df=afr_data_df.assign(o_and_m_plant_2600_sped=o_and_m_plant_2600_sped)
afr_data_df=afr_data_df.assign(o_and_m_plant_2600_reg_sped=o_and_m_plant_2600_reg_sped)
afr_data_df=afr_data_df.assign(regular_exp_all=regular_exp_all)
afr_data_df=afr_data_df.assign(sped_exp_all=sped_exp_all)

afr_data_df=afr_data_df.assign(building_related_expenses=building_related_expenses)

afr_data_df=afr_data_df.assign(salaries_6100_exp=salaries_6100_exp)
afr_data_df=afr_data_df.assign(benefits_6200_exp=benefits_6200_exp)
afr_data_df=afr_data_df.assign(all_salaries_benefits_6100_6200=all_salaries_benefits_6100_6200)
afr_data_df=afr_data_df.assign(purch_serv_6300_6400_6500_exp=purch_serv_6300_6400_6500_exp)
afr_data_df=afr_data_df.assign(supplies_6600_exp=supplies_6600_exp)
afr_data_df=afr_data_df.assign(other_6800_exp=other_6800_exp)

afr_data_df=afr_data_df.assign(cash_balance=cash_balance)

afr_data_df=afr_data_df.assign(salary_instruction_1000_6100_reg_sped=salary_instruction_1000_6100_reg_sped)
afr_data_df=afr_data_df.assign(benefits_instruction_1000_6200_reg_sped=benefits_instruction_1000_6200_reg_sped)
afr_data_df=afr_data_df.assign(salary_benefits_instruction_1000_6100_6200_reg_sped=salary_benefits_instruction_1000_6100_6200_reg_sped)
afr_data_df=afr_data_df.assign(salary_benefits_instruction_1000_6100_6200_reg_sped_class_site=salary_benefits_instruction_1000_6100_6200_reg_sped_class_site)

afr_data_df=afr_data_df.assign(total_instruction_related_personnel=total_instruction_related_personnel)

afr_data_df=afr_data_df.assign(total_students=total_students)
afr_data_df=afr_data_df.assign(certified_teachers=certified_teachers)
afr_data_df=afr_data_df.assign(non_certified_teachers=non_certified_teachers)
afr_data_df=afr_data_df.assign(contract_teachers=contract_teachers)
afr_data_df=afr_data_df.assign(total_teachers=total_teachers)
afr_data_df=afr_data_df.assign(avg_teacher_salary=avg_teacher_salary)
afr_data_df=afr_data_df.assign(avg_teacher_salary_infl=avg_teacher_salary_infl)

### Compute Relevant Ratios

In [33]:
# total expenses to revenue ratio
exp_to_rev_ratio = total_expenses.divide(total_revenue)


# instructional salaries and benefits related ratios
instruction_1000_per_total_exp = instruction_1000_reg_sped.divide(total_expenses)
regular_salary_instruction_1000_per_total_exp = instruction_1000_regular.divide(total_expenses)
salaries_6100_per_total_exp = salaries_6100_exp.divide(total_expenses)
salary_instruction_1000_6100_per_total_exp = salary_instruction_1000_6100_reg_sped.divide(total_expenses)

benefits_6200_per_total_exp = benefits_6200_exp.divide(total_expenses)
benefits_1000_6200_per_total_exp = benefits_instruction_1000_6200_reg_sped.divide(total_expenses)

salary_benefits_instruction_1000_6100_6200_per_total_exp = salary_benefits_instruction_1000_6100_6200_reg_sped.divide(total_expenses)
salary_benefits_instruction_1000_6100_6200_class_site_per_total_exp = salary_benefits_instruction_1000_6100_6200_reg_sped_class_site.divide(total_expenses)
total_instruction_related_personnel_per_total_exp = total_instruction_related_personnel.divide(total_expenses)

benefits_to_salary_ratio_6100_6200 = benefits_6200_exp.divide(salaries_6100_exp)
benefits_to_salary_ratio_1000_6100_6200 = benefits_instruction_1000_6200_reg_sped.divide(salary_instruction_1000_6100_reg_sped)


# administrative expense ratios
support_serv_general_admin_2300_reg_sped_per_total_exp = support_serv_general_admin_2300_reg_sped.divide(total_expenses)
support_serv_school_admin_2400_reg_sped_per_total_exp = support_serv_school_admin_2400_reg_sped.divide(total_expenses)
support_serv_central_serv_2500_reg_sped_per_total_exp = support_serv_central_serv_2500_reg_sped.divide(total_expenses)
admin_expenses_all_per_total_exp = admin_expenses_all.divide(total_expenses)
instructional_exp_to_admin_exp_ratio = instruction_1000_reg_sped.divide(admin_expenses_all)


# non-instruction expense ratios
o_and_m_plant_2600_per_total_exp = o_and_m_plant_2600_reg_sped.divide(total_expenses)
purch_serv_6300_6400_6500_per_total_exp = purch_serv_6300_6400_6500_exp.divide(total_expenses)
supplies_6600_per_total_exp = supplies_6600_exp.divide(total_expenses)
other_6800_per_total_exp = other_6800_exp.divide(total_expenses)
building_related_expenses_per_total_exp = building_related_expenses.divide(total_expenses)


# sped-related and ell-related ratios
instruction_1000_regular_to_sped_ratio = instruction_1000_regular.divide(instruction_1000_sped)
regular_to_sped_instructional_related_ratio = regular_exp_all.divide(sped_exp_all)
sped_exp_all_per_total_exp = sped_exp_all.divide(total_expenses)
ell_exp_per_total_exp = ell_exp.divide(total_expenses)

# cash ratio
days_cash = cash_balance.divide(total_expenses) * 365


# student-teacher ratios
students_per_teacher = total_students.divide(total_teachers)
teachers_per_student = total_teachers.divide(total_students)

# teacher salary proxy
instruction_1000_reg_sped_per_teacher = instruction_1000_reg_sped.divide(total_teachers)

# instructional expenses per student - DISTRICTS
total_instruction_related_personnel_per_student = total_instruction_related_personnel.divide(total_students)

# instructional expenses per student - CHARTERS
salary_benefits_instruction_1000_6100_6200_class_site_per_student = salary_benefits_instruction_1000_6100_6200_reg_sped_class_site.divide(total_students)


### Make DataFrame with Relevant Ratios

In [34]:
# initialize dataframe
ratios_df=pd.DataFrame()

# ctds id and year
ratios_df=ratios_df.assign(ctds_id=ctds_id)
ratios_df=ratios_df.assign(year_b=year)

# percentage of FRL population
ratios_df=ratios_df.assign(pct_free_reduced=pct_free_reduced)

# percentage 'white' population
ratios_df=ratios_df.assign(pct_white=pct_white)

# percentage of 'non_white' population
ratios_df=ratios_df.assign(pct_non_white=pct_non_white)

# percentage of 'sped' population
ratios_df=ratios_df.assign(pct_swd=pct_swd)

# total expenses to revenue ratio
ratios_df=ratios_df.assign(exp_to_rev_ratio=exp_to_rev_ratio) 

# instructional salaries and benefits related ratios
ratios_df=ratios_df.assign(instruction_1000_per_total_exp=instruction_1000_per_total_exp) 
ratios_df=ratios_df.assign(regular_salary_instruction_1000_per_total_exp=regular_salary_instruction_1000_per_total_exp)
ratios_df=ratios_df.assign(salaries_6100_per_total_exp=salaries_6100_per_total_exp) 
ratios_df=ratios_df.assign(salary_instruction_1000_6100_per_total_exp=salary_instruction_1000_6100_per_total_exp)

ratios_df=ratios_df.assign(benefits_6200_per_total_exp=benefits_6200_per_total_exp)
ratios_df=ratios_df.assign(benefits_1000_6200_per_total_exp=benefits_1000_6200_per_total_exp)

ratios_df=ratios_df.assign(salary_benefits_instruction_1000_6100_6200_per_total_exp=salary_benefits_instruction_1000_6100_6200_per_total_exp) 
ratios_df=ratios_df.assign(salary_benefits_instruction_1000_6100_6200_class_site_per_total_exp=salary_benefits_instruction_1000_6100_6200_class_site_per_total_exp) 
ratios_df=ratios_df.assign(total_instruction_related_personnel_per_total_exp=total_instruction_related_personnel_per_total_exp)

ratios_df=ratios_df.assign(benefits_to_salary_ratio_6100_6200=benefits_to_salary_ratio_6100_6200)
ratios_df=ratios_df.assign(benefits_to_salary_ratio_1000_6100_6200=benefits_to_salary_ratio_1000_6100_6200)


# administrative expense ratios
ratios_df=ratios_df.assign(support_serv_general_admin_2300_reg_sped_per_total_exp=support_serv_general_admin_2300_reg_sped_per_total_exp)
ratios_df=ratios_df.assign(support_serv_school_admin_2400_reg_sped_per_total_exp=support_serv_school_admin_2400_reg_sped_per_total_exp) 
ratios_df=ratios_df.assign(support_serv_central_serv_2500_reg_sped_per_total_exp=support_serv_central_serv_2500_reg_sped_per_total_exp)
ratios_df=ratios_df.assign(admin_expenses_all_per_total_exp=admin_expenses_all_per_total_exp)
ratios_df=ratios_df.assign(instructional_exp_to_admin_exp_ratio=instructional_exp_to_admin_exp_ratio)


# non-instruction expense ratios
ratios_df=ratios_df.assign(o_and_m_plant_2600_per_total_exp=o_and_m_plant_2600_per_total_exp)
ratios_df=ratios_df.assign(purch_serv_6300_6400_6500_per_total_exp=purch_serv_6300_6400_6500_per_total_exp)
ratios_df=ratios_df.assign(supplies_6600_per_total_exp=supplies_6600_per_total_exp)
ratios_df=ratios_df.assign(other_6800_per_total_exp=other_6800_per_total_exp)
ratios_df=ratios_df.assign(building_related_expenses_per_total_exp=building_related_expenses_per_total_exp)


# sped-related and ell-related ratios
ratios_df=ratios_df.assign(instruction_1000_regular_to_sped_ratio=instruction_1000_regular_to_sped_ratio)
ratios_df=ratios_df.assign(regular_to_sped_instructional_related_ratio=regular_to_sped_instructional_related_ratio)
ratios_df=ratios_df.assign(sped_exp_all_per_total_exp=sped_exp_all_per_total_exp)
ratios_df=ratios_df.assign(ell_exp_per_total_exp=ell_exp_per_total_exp)


# cash ratio
ratios_df=ratios_df.assign(days_cash=days_cash)


# student-teacher ratios
ratios_df=ratios_df.assign(students_per_teacher=students_per_teacher) 
ratios_df=ratios_df.assign(teachers_per_student=teachers_per_student)


# teacher salary proxy
ratios_df=ratios_df.assign(instruction_1000_reg_sped_per_teacher=instruction_1000_reg_sped_per_teacher)


# instructional expenses per student - DISTRICTS
ratios_df=ratios_df.assign(total_instruction_related_personnel_per_student = total_instruction_related_personnel_per_student)
# instructional expenses per student - CHARTERS
ratios_df=ratios_df.assign(salary_benefits_instruction_1000_6100_6200_class_site_per_student = salary_benefits_instruction_1000_6100_6200_class_site_per_student)

### Export Data to CSV

In [35]:
# ensure column names are not same between df's
afr_data_df = afr_data_df.rename(columns={'ctds_id': 'ctds_id_x'})
ratios_df = ratios_df.rename(columns={'ctds_id': 'ctds_id__xx'})

In [36]:
# export data
export_path = './jupyter_data/'
afr_data_df.to_csv(export_path+'afr_data_df.csv', index_label=False)
ratios_df.to_csv(export_path+'ratios_df.csv', index_label=False)